In [ ]:
#installer pyspark dans mon environnement virtuel
    #!pip install pyspark
    #!pip install findspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, monotonically_increasing_id
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ETL OMOP").getOrCreate()

#recuperer les données
specialite_info = spark.read.csv("ir_spe_v.csv", header=True, inferSchema=True)
actes_infos = spark.read.csv("ir_act_v.csv", header=True, inferSchema=True)

#creer une table unifiée des spécialités, par l'union 
provider_tab = specialite_info.select(
    col("pfs_spe_cod").alias("spe_cod"),
    col("label")
).union(
    actes_infos.select(
        col("pfs_act_nat").alias("spe_cod"),
        col("label")
    )
)

# gerer provider_id par des chiffres auto-incrémentés et je les ajoute à la table
#les autres tables aussi
window = Window.orderBy("spe_cod")
provider_tab = provider_tab.withColumn("provider_id", row_number().over(window))
provider_tab = provider_tab.withColumn("provider_source_value", col("spe_cod"))

# specialty_source_value sera ma liste de specialistes
## Hypothese : le type de la colonne est varchar, et selon la documentation OMOP 
###il peut s'agir soit d'un code du spécialiste, soit le texte de description
provider_tab = provider_tab.withColumn("specialty_source_value", col("label"))

#specialty_concept_id, je pars du principe qu'il manque des données de "sourceCode", j'impute la valeur 0
provider_tab = provider_tab.withColumn("specialty_concept_id", lit(0).cast("integer"))

#créer la table provider
provider = provider_tab.select("provider_id", "specialty_source_value", "specialty_concept_id", "provider_source_value")

# Sauvegarder au format Parquet
provider.write.mode("overwrite").parquet("provider_table.parquet")

# Afficher quelques lignes pour vérifier
provider.show(10, truncate=False)
# Fermer Spark
spark.stop()
